<a href="https://colab.research.google.com/github/gkoorsen/Automatic_multi_docking/blob/main/Smina_Multi_Docking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Instructions:

1. Set the exhaustiveness parameter
   * 8 for faster calculations
   * 16 for slower, but more accurate results

2. Upload the docking template. You can find the template here:
    https://github.com/gkoorsen/Automatic_multi_docking/blob/main/Docking_template.xlsx

3. That's it! Run the cells in the rest of the notebook to proceed with the
  docking with SMINA.


In [8]:
exhaustiveness = 8#@param {type:"integer"}
email_address = "g.koorsen@gmail.com" #@param {type:"string"}

In [ ]:
password = getpass.getpass("Enter your password: ")

#Run the cells below to start docking...

##Install the necessary packages and libraries

In [9]:
!pip install biopython
!pip install wget
!pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 50.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=b82385641bea38dbafb9499e64ee8e4129136393d19c9b1effb107d35ca35c35
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
ERROR: Operation cancelled by user
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
!apt-get -qq install -y cmake
!apt-get -qq install -y swig
!apt-get -qq install -y libeigen3-dev



Selecting previously unselected package swig4.0.
(Reading database ... 123069 files and directories currently installed.)
Preparing to unpack .../swig4.0_4.0.1-5build1_amd64.deb ...
Unpacking swig4.0 (4.0.1-5build1) ...
Selecting previously unselected package swig.
Preparing to unpack .../swig_4.0.1-5build1_all.deb ...
Unpacking swig (4.0.1-5build1) ...
Setting up swig4.0 (4.0.1-5build1) ...
Setting up swig (4.0.1-5build1) ...
Processing triggers for man-db (2.9.1-1) ...
Selecting previously unselected package libeigen3-dev.
(Reading database ... 123820 files and directories currently installed.)
Preparing to unpack .../libeigen3-dev_3.3.7-2_all.deb ...
Unpacking libeigen3-dev (3.3.7-2) ...
Setting up libeigen3-dev (3.3.7-2) ...


In [ ]:
!wget https://github.com/openbabel/openbabel/archive/openbabel-2-4-1.tar.gz
!tar xzvf openbabel-2-4-1.tar.gz


In [ ]:
!mkdir openbabel-openbabel-2-4-1/build
%cd openbabel-openbabel-2-4-1/build
!cmake ../ -DPYTHON_BINDINGS=ON -DRUN_SWIG=ON
!make
!make install
%cd /content


/content/openbabel-openbabel-2-4-1/build
CMake Deprecation Warning at CMakeLists.txt:3 (cmake_minimum_required):
  Compatibility with CMake < 2.8.12 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


-- The C compiler identification is GNU 9.4.0
-- The CXX compiler identification is GNU 9.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
CMake Deprecation Warning at CMakeLists.txt:16 (cmake_policy):
  The OLD behavior for policy CMP0005 will be removed from a future version
  of CM

In [ ]:
import os
os.environ['LD_LIBRARY_PATH'] += ':/usr/local/lib'


In [ ]:
!wget https://sourceforge.net/projects/smina/files/smina.static/download -O smina && chmod +x smina


In [ ]:
from Bio.PDB import *
import subprocess
import re
import pandas as pd
import requests
import time
from tqdm import tqdm
import os
from google.colab import files, drive
from numpy import mean
from numpy import amin, amax, array
import numpy as np
import smtplib
from email.mime.text import MIMEText
import getpass


## Define functions

In [ ]:
class NoHeteroSelect(Select):
    def accept_residue(self, residue):
        return 1 if residue.get_id()[0] == " " else 0

from Bio.PDB import Chain

from Bio.PDB import PDBParser, PDBIO, Select
from Bio.PDB.Entity import Entity



def prepare_structure(name, file, ligand_res_name):

  with open(file, 'r') as f:
    lines = f.readlines()

  with open(f'{name}_ligand.pdb','w') as o:
    with open(f'{name}_apo.pdb','w') as a:
      for line in lines:
        if line.find('HETATM') != -1:
          if line.find(ligand_res_name) != -1:
            o.write(line)
        else:
          a.write(line)

  command = f'obabel {name}_ligand.pdb -O {name}_ligand.pdbqt -h'
  process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
  process.wait()

  command = f'obabel {name}_apo.pdb -O {name}_apo_prepared.pdbqt -h'
  process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
  process.wait()


def prepare_ligand(name, smiles):

    if smiles is not None:

        command = f'obabel -:\"{smiles}\" -O {name}.sdf --gen2D'
        process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
        process.wait()


        command = f'obabel {name}.sdf -O {name}_3d.sdf --gen3D'
        process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
        process.wait()

        command = f'obabel {name}_3d.sdf -O {name}.pdbqt'
        process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
        process.wait()

        # Print the output
        print(process.stdout.read().decode())

    else:
        print('SMILES NOT FOUND')



def dock(name, ligand_name, smiles,exhaustiveness):

    prepare_ligand(ligand_name,smiles)

    # Define the command as a string
    command = f"/content/smina -r {name}_apo_prepared.pdbqt -l {ligand_name}.pdbqt --autobox_ligand {name}_ligand.pdbqt --autobox_add 8 --exhaustiveness {exhaustiveness} -o {name}_result.pdbqt"
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
    process.wait()

    command = f"grep '^REMARK minimizedAffinity' {name}_result.pdbqt | head -n 1 | awk '{{print $3}}'"
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
    process.wait()
    top_score = subprocess.check_output(command, shell=True)
    top_score = top_score.decode("utf-8").strip()

    return top_score


def download_pdb_file(pdb_id: str) -> str:

    PDB_DIR ="/tmp/pdb/"
    os.makedirs(PDB_DIR, exist_ok=True)

    # url or pdb_id
    if pdb_id.startswith('http'):
        url = pdb_id
        filename = url.split('/')[-1]
    elif pdb_id.endswith(".pdb"):
        return pdb_id
    else:
        if pdb_id.startswith("AF"):
            url = f'https://alphafold.ebi.ac.uk/files/{pdb_id}-model_v3.pdb'
        else:
            url = f'http://files.rcsb.org/view/{pdb_id}.pdb'
        filename = f'{pdb_id}.pdb'

    cache_path = os.path.join(PDB_DIR, filename)
    if os.path.exists(cache_path):
        return cache_path

    pdb_req = requests.get(url)
    pdb_req.raise_for_status()
    open(cache_path, 'w').write(pdb_req.text)
    return cache_path


In [ ]:
def send_email(user, pwd, recipient, subject, body):
    gmail_user = user
    gmail_pwd = pwd
    FROM = user
    TO = recipient if type(recipient) is list else [recipient]
    SUBJECT = subject
    TEXT = body

    # Prepare actual message
    message = """From: %s\nTo: %s\nSubject: %s\n\n%s
    """ % (FROM, ", ".join(TO), SUBJECT, TEXT)
    try:
        server = smtplib.SMTP("smtp.gmail.com", 587)
        server.ehlo()
        server.starttls()
        server.login(gmail_user, gmail_pwd)
        server.sendmail(FROM, TO, message)
        server.close()
        print('Successfully sent the mail')
    except Exception as e:
        print("Failed to send mail:", str(e))


##Upload Excel template file

In [ ]:
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
df = pd.read_excel(file_name)


In [ ]:
pdb_dict = {p : l for p,l in zip(df.dropna()['PDBs'],df.dropna()['Ligands'])}
smiles = df['SMILES']
names = [f'Compound_{i}' for i in range(len(smiles))]
compound2name = {c : n for c,n in zip(names,df['Compound names'])}

In [ ]:
PDB_files = [download_pdb_file(p) for p in tqdm(pdb_dict.keys())]

##Perform Docking

In [ ]:


drive.mount('/content/gdrive')
top_affinities = {}

for (n,l) in pdb_dict.items():
    print(f'\t Preparing receptor {n}....')
    p = f'/tmp/pdb/{n}.pdb'
    prepare_structure(n,p,l)
    for m,s,c in zip(names,smiles,df['Compound names']):
        if s != None:
            top_affinities[(n,m)] = dock(n,m,s,8)
            print(f'-{c}: {top_affinities[(n,m)]}')


send_email(email_address, password, email_address, "SMINA notebook finished", "Check the results")

with open('/content/gdrive/MyDrive/smina_results.pk','wb') as f:
  pickle.dump(top_affinities,f)

